In [3]:
!pip install psycopg2-binary elasticsearch scikit-learn

Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.15.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl (11.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.15.2-cp312-cp312-win_amd64.whl (40.9 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


# Product Recommendation System
This notebook implements a recommendation system using PostgreSQL for product data and Elasticsearch for user history.

In [5]:
import sys
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import string
import psycopg2
from elasticsearch import Elasticsearch

DB_PARAMS = {
    "host": "localhost",
    "database": "ShopMartDB",
    "user": "postgres",
    "password": "postgres",
    "port": "5432"
}

# Elasticsearch connection
ES_CLIENT = Elasticsearch("http://localhost:9200")

## Database Connection Functions

In [7]:
def get_db_connection():
    return psycopg2.connect(**DB_PARAMS)

def load_data_from_postgres():
    conn = get_db_connection()
    query = """
        SELECT asin, title, price, rating, category, discount, reviews_count, prime
        FROM amazon_products
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

## Data Preprocessing Functions

In [9]:
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ' '.join(text.split())
    simple_stopwords = {
        'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while',
    }
    text = ' '.join([word for word in text.split() if word not in simple_stopwords])
    return text

def preprocess_data(df):
    df = df.copy()
    df['discount'] = df['discount'].replace("No Discount", 0)
    df['discount'] = pd.to_numeric(df['discount'], errors='coerce')
    df['discount'] = df['discount'].fillna(0)
    if df['price'].dtype == object:
        df['price'] = df['price'].replace('[\₹,$,£,€,]', '', regex=True).astype(str).str.replace(',', '')
        df['price'] = pd.to_numeric(df['price'], errors='coerce')
    df['price'] = df['price'].fillna(df['price'].median())
    df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
    df['rating'] = df['rating'].fillna(df['rating'].median())
    df['reviews_count'] = pd.to_numeric(df['reviews_count'], errors='coerce')
    df['reviews_count'] = df['reviews_count'].fillna(0)
    df['processed_title'] = df['title'].apply(preprocess_text)
    return df

## Feature Engineering Functions

In [10]:
def create_content_representation(df):
    def create_feature_soup(row):
        features = []
        prime_status = 'prime' if row.get('prime') == 1 else 'not_prime'
        features.append(prime_status)
        if isinstance(row.get('category'), str):
            category = row.get('category').lower().replace(' ', '_')
            features.append(f"category_{category}")
        if row['price'] < 500:
            price_range = 'budget_price'
        elif row['price'] < 7500:
            price_range = 'mid_price'
        else:
            price_range = 'premium_price'
        features.append(price_range)
        if row['rating'] >= 4.5:
            rating_range = 'top_rated'
        elif row['rating'] >= 4.0:
            rating_range = 'highly_rated'
        elif row['rating'] >= 3.0:
            rating_range = 'average_rated'
        else:
            rating_range = 'low_rated'
        features.append(rating_range)
        if row['discount'] >= 30:
            discount_range = 'high_discount'
        elif row['discount'] >= 10:
            discount_range = 'medium_discount'
        else:
            discount_range = 'low_discount'
        features.append(discount_range)
        if row['reviews_count'] > 1000:
            popularity = 'very_popular'
        elif row['reviews_count'] > 100:
            popularity = 'popular'
        else:
            popularity = 'less_popular'
        features.append(popularity)
        return ' '.join(features)
    df['feature_soup'] = df.apply(create_feature_soup, axis=1)
    return df

def extract_camera_terms(title):
    if not isinstance(title, str):
        return ""
    camera_terms = [
        'dslr', 'mirrorless', 'canon', 'nikon', 'sony', 'fujifilm', 'olympus', 'panasonic',
    ]
    title_lower = title.lower()
    found_terms = []
    for term in camera_terms:
        if term in title_lower:
            found_terms.append(term)
    model_patterns = [
        r'\b[a-z]+\d+[a-z]*\b',
        r'\b\d+[a-z]+\b',
        r'\bmark\s+[ivx]+\b',
    ]
    for pattern in model_patterns:
        matches = re.findall(pattern, title_lower)
        found_terms.extend(matches)
    return ' '.join(found_terms)

## Recommendation Model

In [12]:
def build_recommendation_model(df):
    if 'processed_title' not in df.columns:
        df['processed_title'] = df['title'].apply(preprocess_text)
    df['camera_terms'] = df['title'].apply(extract_camera_terms)
    title_tfidf = TfidfVectorizer(
        max_features=5000,
        stop_words='english',
        ngram_range=(1, 2)
    )
    title_tfidf_matrix = title_tfidf.fit_transform(df['processed_title'])
    camera_tfidf = TfidfVectorizer(stop_words='english')
    if df['camera_terms'].str.strip().str.len().sum() > 0:
        camera_tfidf_matrix = camera_tfidf.fit_transform(df['camera_terms'])
        has_camera_terms = True
    else:
        camera_tfidf_matrix = title_tfidf_matrix.copy()
        has_camera_terms = False
    feature_tfidf = TfidfVectorizer(stop_words='english')
    feature_tfidf_matrix = feature_tfidf.fit_transform(df['feature_soup'])
    title_sim = cosine_similarity(title_tfidf_matrix, title_tfidf_matrix)
    feature_sim = cosine_similarity(feature_tfidf_matrix, feature_tfidf_matrix)
    if has_camera_terms:
        camera_sim = cosine_similarity(camera_tfidf_matrix, camera_tfidf_matrix)
        combined_sim = 0.5 * title_sim + 0.3 * camera_sim + 0.2 * feature_sim
    else:
        combined_sim = 0.7 * title_sim + 0.3 * feature_sim
    return combined_sim

def get_recommendations(product_idx, cosine_sim, df, top_n=5):
    try:
        if product_idx < 0 or product_idx >= len(df):
            print(f"Product index {product_idx} is out of range (0-{len(df)-1}).")
            return pd.DataFrame()
        sim_scores = list(enumerate(cosine_sim[product_idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:top_n+1]
        product_indices = [i[0] for i in sim_scores]
        columns_to_include = ['asin', 'title', 'price', 'rating', 'category', 'discount', 'reviews_count']
        available_columns = [col for col in columns_to_include if col in df.columns]
        recommendations = df.iloc[product_indices][available_columns].copy()
        recommendations['similarity_score'] = [score for _, score in sim_scores]
        return recommendations
    except Exception as e:
        print(f"Error getting recommendations: {e}")
        return pd.DataFrame()

## History-based Recommendations

In [15]:
def get_recommendations_from_history(user_history, df, top_n=9):
    if not user_history or len(user_history) == 0:
        return pd.DataFrame()
    processed_df = preprocess_data(df)
    content_df = create_content_representation(processed_df)
    combined_sim = build_recommendation_model(content_df)
    history_indices = []
    for asin in user_history:
        indices = content_df.index[content_df['asin'] == asin].tolist()
        if indices:
            history_indices.append(indices[0])
    if not history_indices:
        return pd.DataFrame()
    all_recommendations = pd.DataFrame()
    for idx in history_indices:
        recommendations = get_recommendations(idx, combined_sim, content_df, top_n)
        if all_recommendations.empty:
            all_recommendations = recommendations
        else:
            all_recommendations = pd.concat([all_recommendations, recommendations])
    all_recommendations = all_recommendations.sort_values('similarity_score', ascending=False)
    all_recommendations = all_recommendations.drop_duplicates(subset='asin')
    return all_recommendations.head(top_n)

def get_user_history_from_elasticsearch(user_id):
    query = {
        "query": {
            "term": {
                "user_id": user_id
            }
        }
    }
    response = ES_CLIENT.search(index="user_history", body=query)
    history = [hit["_source"]["product_asin"] for hit in response["hits"]["hits"]]
    return history

## Example Usage

In [16]:
# Load data
df = load_data_from_postgres()

# Example 1: Product-based recommendations
asin = "B08J5W9C9Q"
top_n = 5
product_idx = df.index[df['asin'] == asin].tolist()
if product_idx:
    processed_df = preprocess_data(df)
    content_df = create_content_representation(processed_df)
    combined_sim = build_recommendation_model(content_df)
    recommendations = get_recommendations(product_idx[0], combined_sim, content_df, top_n)
    display(recommendations)

# Example 2: History-based recommendations
user_id = "user123"
user_history = get_user_history_from_elasticsearch(user_id)
history_recommendations = get_recommendations_from_history(user_history, df, top_n=5)
display(history_recommendations)

C:\Users\Abhirat\AppData\Local\Temp\ipykernel_22248\2824819319.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


DatabaseError: Execution failed on sql '
        SELECT asin, title, price, rating, category, discount, reviews_count, prime
        FROM amazon_products
    ': column "reviews_count" does not exist
LINE 2: ...T asin, title, price, rating, category, discount, reviews_co...
                                                             ^
